In [7]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv


In [0]:
import zipfile
import io
data = zipfile.ZipFile(io.BytesIO(uploaded["train-scene classification.zip"]), 'r')
data.extractall()

In [1]:
!ls

 sample_data   train	  'train-scene classification (1).zip'	 vgg16.h5
 test.csv      train.csv  'train-scene classification.zip'


In [0]:
!cd 

In [2]:
from keras.models import Sequential
from scipy.misc import imread
get_ipython().magic('matplotlib inline')
import matplotlib.pyplot as plt
import numpy as np
import keras
from keras.layers import Dense
import pandas as pd
from scipy.misc import imresize
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np
from keras.applications.vgg16 import decode_predictions

Using TensorFlow backend.


In [0]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
train_path="train/"

In [12]:
train.head()

,image_name,label
0,0.jpg,0
1,1.jpg,4
2,2.jpg,5
3,4.jpg,0
4,7.jpg,4


In [26]:
test.head()

,image_name
0,3.jpg
1,5.jpg
2,6.jpg
3,11.jpg
4,14.jpg


In [0]:
train_img=[]
for i in range(len(train[0:10000])):

    temp_img=image.load_img(train_path+train['image_name'][i],target_size=(224,224))

    temp_img=image.img_to_array(temp_img)

    train_img.append(temp_img)

In [0]:
#converting train images to array and applying mean subtraction processing

train_img=np.array(train_img) 
train_img=preprocess_input(train_img)

In [0]:
test_img=[]
for i in range(len(test)):

    temp_img=image.load_img(train_path+test['image_name'][i],target_size=(224,224))

    temp_img=image.img_to_array(temp_img)

    test_img.append(temp_img)


In [0]:
test_img=np.array(test_img) 
test_img=preprocess_input(test_img)

In [0]:
model = VGG16(weights='imagenet', include_top=False)

In [0]:
features_test=model.predict(test_img)

In [10]:
features_train=model.predict(train_img)

NameError: ignored

In [7]:
features_train.shape

(10000, 7, 7, 512)

In [16]:
features_test.shape

(7301, 7, 7, 512)

In [18]:
train_x=features_train.reshape(10000,25088)

ValueError: ignored

In [0]:
test_x=features_test.reshape(7301,25088)

In [0]:
train_y=np.asarray(train['label'][0:10000])

In [0]:
train_y=pd.get_dummies(train_y)
train_y=np.array(train_y)

In [11]:
train_y[0]

array([1, 0, 0, 0, 0, 0], dtype=uint8)

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_valid, Y_train, Y_valid=train_test_split(train_x,train_y,test_size=0.3, random_state=42)

In [12]:
X_train[0]

NameError: ignored

In [0]:
from keras.layers import Dense, Activation
model=Sequential()

model.add(Dense(1000, input_dim=25088, activation='relu',kernel_initializer='uniform'))
keras.layers.core.Dropout(0.3, noise_shape=None, seed=None)

model.add(Dense(500,input_dim=1000,activation='sigmoid'))
keras.layers.core.Dropout(0.4, noise_shape=None, seed=None)

model.add(Dense(150,input_dim=500,activation='sigmoid'))
keras.layers.core.Dropout(0.2, noise_shape=None, seed=None)

model.add(Dense(units=6))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [14]:
# fitting the model 

model.fit(X_train, Y_train, epochs=30, batch_size=128,validation_data=(X_valid,Y_valid))

Instructions for updating:
Use tf.cast instead.
Train on 7000 samples, validate on 3000 samples
Epoch 1/30
7000/7000 [==============================] - 4s 594us/step - loss: 0.4583 - acc: 0.8546 - val_loss: 0.2535 - val_acc: 0.9137
Epoch 2/30
7000/7000 [==============================] - 3s 393us/step - loss: 0.1663 - acc: 0.9487 - val_loss: 0.2234 - val_acc: 0.9193
Epoch 3/30
7000/7000 [==============================] - 3s 393us/step - loss: 0.1031 - acc: 0.9691 - val_loss: 0.2187 - val_acc: 0.9200
Epoch 4/30
7000/7000 [==============================] - 3s 399us/step - loss: 0.0580 - acc: 0.9856 - val_loss: 0.2301 - val_acc: 0.9197
Epoch 5/30
7000/7000 [==============================] - 3s 396us/step - loss: 0.0305 - acc: 0.9929 - val_loss: 0.2568 - val_acc: 0.9200
Epoch 6/30
7000/7000 [==============================] - 3s 396us/step - loss: 0.0141 - acc: 0.9976 - val_loss: 0.2767 - val_acc: 0.9190
Epoch 7/30
7000/7000 [==============================] - 3s 397us/step - loss: 0.0083 - a

In [0]:
from keras.models import load_model

model.save('vgg16.h5') 

In [5]:
new_model = keras.models.load_model('vgg16.h5')
new_model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1000)              25089000  
_________________________________________________________________
dense_2 (Dense)              (None, 500)               500500    
_________________________________________________________________
dense_3 (Dense)              (None, 150)               75150     
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 906       
_________________________________________________________________
activation_1 (Activation)    (None, 6)                 0         
Total params: 25,665,556
Trainable params: 25,665,556
Non-trainable params: 0
_________________________________________________________________


In [0]:
y_pred=new_model.predict(test_x, batch_size=128)

In [21]:
y_pred[0]

array([3.3010732e-05, 2.6304349e-05, 1.9085095e-05, 2.9051358e-05,
       1.7387176e-05, 9.9987519e-01], dtype=float32)

In [0]:
y_classes = y_pred.argmax(axis=-1)

In [23]:
y_classes[0]

5

In [0]:
prediction=pd.DataFrame()

In [0]:
prediction["image_name"]=test["image_name"]

In [0]:
prediction["label"] = y_classes

In [32]:
prediction.head()

,image_name,label
0,3.jpg,5
1,5.jpg,0
2,6.jpg,4
3,11.jpg,2
4,14.jpg,5


In [0]:
prediction.to_csv('intel_prediction0.csv',index=False)

In [0]:
from google.colab import files
files.download("intel_prediction0.csv")


In [0]:
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation

from sklearn.metrics import log_loss

In [0]:
from keras.models import Model
def vgg16_model(img_rows, img_cols, channel=1, num_classes=None):

    model = VGG16(weights='imagenet', include_top=True)

    model.layers.pop()

    model.outputs = [model.layers[-1].output]

    model.layers[-1].outbound_nodes = []

    x=Dense(num_classes, activation='softmax')(model.output)

    model=Model(model.input,x)
    for layer in model.layers[:8]:

       layer.trainable = False
       
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [0]:
img_rows, img_cols = 224, 224 # Resolution of inputs
channel = 3
num_classes = 6 
batch_size = 32 
nb_epoch = 30

In [0]:
model1 = vgg16_model(img_rows, img_cols, channel, num_classes)

In [30]:
model.fit(X_train, Y_train,batch_size=batch_size,epochs=nb_epoch,shuffle=True,verbose=1,validation_data=(X_valid, Y_valid))

Train on 5600 samples, validate on 2400 samples
Epoch 1/30
5600/5600 [==============================] - 7s 1ms/step - loss: 0.1495 - acc: 0.9486 - val_loss: 0.2908 - val_acc: 0.9025
Epoch 2/30
5600/5600 [==============================] - 6s 1ms/step - loss: 0.1411 - acc: 0.9512 - val_loss: 0.2714 - val_acc: 0.9046
Epoch 3/30
5600/5600 [==============================] - 6s 1ms/step - loss: 0.1322 - acc: 0.9555 - val_loss: 0.2542 - val_acc: 0.9158
Epoch 4/30
5600/5600 [==============================] - 6s 1ms/step - loss: 0.1399 - acc: 0.9541 - val_loss: 0.2799 - val_acc: 0.9004
Epoch 5/30
5600/5600 [==============================] - 6s 1ms/step - loss: 0.1386 - acc: 0.9518 - val_loss: 0.2822 - val_acc: 0.9062
Epoch 6/30
5600/5600 [==============================] - 6s 1ms/step - loss: 0.1328 - acc: 0.9550 - val_loss: 0.2836 - val_acc: 0.9050
Epoch 7/30
5600/5600 [==============================] - 6s 1ms/step - loss: 0.1232 - acc: 0.9568 - val_loss: 0.2420 - val_acc: 0.9129
Epoch 8/30
560

In [22]:
!ls

 sample_data   train	  'train-scene classification (1).zip'	 vgg16.h5
 test.csv      train.csv  'train-scene classification.zip'
